In [43]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from datetime import datetime, timedelta

In [2]:
path = "linkedin_page/scraps/05/"

qtd_scraps = len([file for file in os.listdir(path) if file.endswith(".html")]) 
soup_list = []
for i in range(qtd_scraps):
    filename = "post_" + str(i) + ".html"
    with open(path + filename, "r", encoding="utf8") as file:
        soup_list.append(BeautifulSoup(file, "html.parser"))

In [3]:
print(len(soup_list))

134


In [4]:
post_columns = [
    "date",
    "shared_post"
    "type",
    "url",
    "text_content",
    "qtd_reactions",
    "qtd_comments",
    "qtd_shares",
]
df = pd.DataFrame(columns=post_columns)

def add_row(df, post_dict):
    new_row = pd.Series(post_dict, index=df.columns)
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return df

def remove_last_row(df):
    df = df.drop(df.index[-1])
    return df

In [42]:
df_dict = {
    "publish_date": [],
    "shared_post": [],
    "author_name": [],
    "author_profile_url": [],
    "post_type": [],
    "post_url": [],
    "post_content": [],
    "reaction_count": [],
    "comment_count": [],
    "share_count": [],
}
corr_index_filename = []


for index, soup in enumerate(soup_list):
    # print(soup.prettify())
    publish_date = (
        soup.find_all(class_="update-components-actor__meta")[0]
        .find_all(class_="visually-hidden")[-1]
        .text.split("•")[0]
        if soup.find_all(class_="update-components-actor__meta") != []
        else None
        if (soup.find(class_="update-components-promo-v1__text-container") != None)
        or (len(soup.find_all(tag="div")) <= 7)
        else "Unknown"
    )

    # Linkedin Promo or Unloaded scraps
    if publish_date == None:
        continue

    shared_post = (
        True
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else False
    )

    author_name = soup.find(class_="update-components-actor__name").span.span.text

    author_profile_url = soup.find(class_="update-components-actor__meta-link")["href"]

    post_type = (
        "Shared Content"
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else "Poll"
        if soup.find(class_="update-components-poll") is not None
        else "Scheduled Event"
        if soup.find(class_="update-components-scheduled-live-content__event-link")
        is not None
        else "Live"
        if soup.find(class_="video-live") is not None
        else "Single Image"
        if soup.find(class_="update-components-image--single-image") is not None
        else "Text only"
        if "update-v2-social-activity"
        in soup.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
        else "Default"
    )

    post_url = (
        "https://www.linkedin.com/feed/update/"
        + soup.find(class_="feed-shared-update-v2")["data-urn"]
    )

    post_content = (
        soup.find(class_="update-components-update-v2__commentary").span.span.text
        if post_type == "Shared Content"
        else soup.find(class_="update-components-text").span.span.text
    )

    # Reaction
    if soup.find(class_="social-details-social-counts__reactions-count") != None:
        reaction_count = soup.find(
            class_="social-details-social-counts__reactions-count"
        ).text.strip()

    else:
        qtd_reactions = soup.find(
            class_="social-details-social-counts__social-proof-text"
        ).text.strip()

        re_pattern_reactions = r"\b(\d+)\s+pessoas\b"

        reaction_count = 1 + int(re.findall(re_pattern_reactions, qtd_reactions)[0])

    # Comments
    comment_count = (
        soup.find(
            class_="social-details-social-counts__comments"
        ).button.span.text.strip().split(' ')[0]
        if soup.find(class_="social-details-social-counts__comments") != None
        else "0"
    )

    # re_pattern_comments = r"\b(\d+)\b" if comment_count != "0" else "0"

    # df_dict["comment_count"].append(re.findall(re_pattern_comments, comment_count)[0])

    # Share
    share_count = (
        0
        if len(
            soup.find_all(class_="social-details-social-counts__item--right-aligned")
        )
        <= 1
        else soup.find_all(class_="social-details-social-counts__item--right-aligned")[
            1
        ]
        .button.span.text.strip()
        .split(" ")[0]
    )

    df_dict["publish_date"].append(publish_date)
    df_dict["shared_post"].append(shared_post)
    df_dict["author_name"].append(author_name)
    df_dict["author_profile_url"].append(author_profile_url)
    df_dict["post_type"].append(post_type)
    df_dict["post_url"].append(post_url)
    df_dict["post_content"].append(post_content)
    df_dict["reaction_count"].append(reaction_count)
    df_dict["comment_count"].append(comment_count)
    df_dict["share_count"].append(share_count)

df = pd.DataFrame(df_dict)

In [55]:
def define_date(raw_date, date):
    num, mult, _ = raw_date.split(' ')
    if mult == 'd':
        publish_date = date - timedelta(days=int(num))
    elif mult == 'sem':
        publish_date = date - timedelta(days=int(num) * 7)
    elif mult == 'm':
        publish_date = date - timedelta(days=int(num) * 30)
    
    return publish_date.date()

today = datetime.now()

df["publish_date"] = df["publish_date"].apply(lambda x: define_date(x, today))
df

,publish_date,shared_post,author_name,author_profile_url,post_type,post_url,post_content,reaction_count,comment_count,share_count
0,2023-12-10,True,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Vemmmmm,10,0,0
1,2023-12-08,True,Camila Igne,https://www.linkedin.com/in/camila-igne?miniPr...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Quando iniciei minha migração de carreira em m...,25,6,1
2,2023-12-07,True,Rubia Macedo,https://www.linkedin.com/in/rubia-macedo?miniP...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,É bem comum ao final do ano que retrospectivas...,13,5,1
3,2023-12-04,True,Vanêssa Santana,https://www.linkedin.com/in/vanessa-patricia?m...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Ontem tivemos a nossa última mensal de 2023 do...,76,9,11
4,2023-12-07,True,Taymara Teixeira,https://www.linkedin.com/in/taymara?miniProfil...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Venha apoiar a SouJunior! 💙 🚀,7,0,0
...,...,...,...,...,...,...,...,...,...,...
129,2023-10-12,True,Daiane Bolzan,https://www.linkedin.com/in/daiane-deponti-bol...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Sábado é dia de reunião do projeto Connectatto...,85,7,3
130,2023-10-12,True,Vanêssa Santana,https://www.linkedin.com/in/vanessa-patricia?m...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,"Mais novidades na SouJunior!Como alguns sabem,...",127,12,9
131,2023-10-12,True,Sabrina Miranda,https://www.linkedin.com/in/sabrina-miranda-in...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,"Olá, Rede. Tudo bem?! Gostaria de compartilhar...",44,1,1
132,2023-10-12,False,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Live,https://www.linkedin.com/feed/update/urn:li:ac...,Live: aprender GOLang 2 !Nos ajude com $: Gith...,18,8,1


In [ ]:
df.to_csv("linkedin_page/dataframes/main_dataframe.csv", index=False)